In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import math
from statsmodels.stats import diagnostic
import statsmodels.stats as st
from scipy import stats
import networkx as nx
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge
from sklearn.ensemble import RandomForestClassifier
# from sklearn.preprocessing import OneHotEncoder
# from pandas.plotting import scatter_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, auc, roc_curve

from director_scrap import director_scrap

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
columns_char = ['Wiki_ID', 'Freebase_ID', 'Movie_release', 'Char_name', 'Actor_birth', 'Actor_gender', 
               'Actor_height', 'Actor_ethnicity', 'Actor_name', 'Actor_age_release', 'Char/Actor_map', 
               'Char_ID', 'Actor_ID']
df_char = pd.read_table('data/character.metadata.tsv', header=None, names=columns_char)
df_char.head(2)

,Wiki_ID,Freebase_ID,Movie_release,Char_name,Actor_birth,Actor_gender,Actor_height,Actor_ethnicity,Actor_name,Actor_age_release,Char/Actor_map,Char_ID,Actor_ID
0,975900,/m/03vyhn,2001-08-24,Akooshay,1958-08-26,F,1.62,NaN,Wanda De Jesus,42.0,/m/0bgchxw,/m/0bgcj3x,/m/03wcfv7
1,975900,/m/03vyhn,2001-08-24,Lieutenant Melanie Ballard,1974-08-15,F,1.78,/m/044038p,Natasha Henstridge,27.0,/m/0jys3m,/m/0bgchn4,/m/0346l4


In [6]:
columns_mov = ['Wiki_ID', 'Freebase_ID', 'Movie_name', 'Movie_release', 'Movie_revenue', 'Movie_runtime', 
               'Movie_languages', 'Movie_countries', 'Movie_genres']
df_mov = pd.read_table('data/movie.metadata.tsv', header=None, names=columns_mov)
df_mov.head(2)

,Wiki_ID,Freebase_ID,Movie_name,Movie_release,Movie_revenue,Movie_runtime,Movie_languages,Movie_countries,Movie_genres
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science..."
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp..."


In [7]:
print(f"Movie : {len(df_mov)}      Actor : {len(df_char)}")

Movie : 81741      Actor : 450669


### When describing the relevant aspects of the data, and any other datasets you may intend to use, you should in particular show (non-exhaustive list):

**-That you can handle the data in its size.**

**-That you understand what’s in the data (formats, distributions, missing values, correlations, etc.).**

**-That you considered ways to enrich, filter, transform the data according to your needs.**

**-That you have a reasonable plan and ideas for methods you’re going to use, giving their essential mathematical details in the notebook.**

**-That your plan for analysis and communication is reasonable and sound, potentially discussing alternatives to your choices that you considered but dropped.**

In [12]:
import csv

wiki_ids = df_mov['Wiki_ID'].values
directors = []
for wiki_id in wiki_ids:
    director = director_scrap(wiki_id)
    directors.append([wiki_id, director])
    
with open('directors.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Wiki_ID', 'Director'])  # Write header
    writer.writerows(directors)

Director of the movie is: John Carpenter
Director not found
Director of the movie is: Sølve Skagen
Director of the movie is: Donald Cammell
Director of the movie is: Robert van Ackeren
Director of the movie is: Henry Lehrman
Director of the movie is: Henry King
Director of the movie is: John Llewellyn Moxey
Director of the movie is: Dorothy Arzner
Director of the movie is: Toshiya Fujita
Director of the movie is: Roberto Benabib
Director of the movie is: Kenneth Branagh
Director of the movie is: Atıf Yılmaz
Director of the movie is: Dave Fleischer
Director of the movie is: Hugh Brody
Director of the movie is: Robert Stevenson
Director of the movie is: Kurt Maetzig
Director of the movie is: Andy Tennant
Director of the movie is: Edmund Goulding
Director of the movie is: Abel Ferrara
Director not found
Director not found
Director of the movie is: Nikolai Ekk
Director not found
Director of the movie is: Gianni Franciolini
Director of the movie is: John Reid
Director of the movie is: John 

KeyboardInterrupt: 